In [ ]:
##remember to change Runtime type to GPU
from google.colab import drive ## give permission to colab to mount to Google Drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My\ Drive/segmentation

import tensorflow as tf
import time

## please specify your wanted batch size here
batch_size = 16 # if the RAM is keeps getting OOM, then please reduce the training batch_size
num_classes=7
image_size = 480*640

# Create a dictionary describing the features.
image_feature_description = {
    'rgb': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
    'x': tf.io.FixedLenFeature([], tf.string),
    'y': tf.io.FixedLenFeature([], tf.string),
    'z': tf.io.FixedLenFeature([], tf.string),
    'weight': tf.io.FixedLenFeature([num_classes],tf.float32),
}

## flip the image horizontally
def flip(image):
    return tf.image.flip_left_right(image)

def subtract(image):
    return tf.math.subtract(tf.ones(tf.shape(image))*255,image)

#to rescale image back to [0,1] range
def rescale(x):
    x = tf.cast(x,tf.float32)
    max_val = tf.reduce_max(x)
    min_val = tf.reduce_min(x)
    return (x-min_val)/(max_val-min_val+1e-7)

##crop the image
def crop(img,height,width,x1,y1):
    img = img[y1:y1+height, x1:x1+width,:]
    return img

#blur the image by sliding a Gaussian kernel accross it
def gaussian_blur(img, kernel_size=7, sigma=5):
    def gauss_kernel(channels, kernel_size, sigma):
        
        ax = tf.range(-tf.cast(tf.math.floordiv(kernel_size,2),tf.int32) + 1, 
                      tf.cast(tf.math.floordiv(kernel_size,2),tf.int32) + 1)  
           
        xx, yy = tf.meshgrid(ax, ax)
        yy = tf.cast(yy,tf.int32)
        kernel = tf.exp(-(tf.cast(xx ** 2,tf.float32) + tf.cast(yy ** 2,tf.float32)) / (2 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
        return kernel

    img = tf.cast(tf.expand_dims(img,axis=0),tf.float32)
    gaussian_kernel = gauss_kernel(tf.shape(img)[-1], kernel_size, sigma)
    gaussian_kernel = gaussian_kernel[..., tf.newaxis]
    gaussian_kernel = tf.cast(gaussian_kernel,tf.float32)
    
    img = tf.nn.depthwise_conv2d(img, gaussian_kernel, [1, 1, 1, 1],
                                  padding='SAME', data_format='NHWC')
    img = tf.cast(tf.squeeze(img),tf.float32)

    return img
    
#parse function for training data
def train_parse(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    data = tf.io.parse_single_example(example_proto, image_feature_description)
    rgb = tf.image.decode_jpeg(data['rgb'])
    label = tf.image.decode_png(data['label'],dtype=tf.uint8)
    x = tf.image.decode_jpeg(data['x'])
    y = tf.image.decode_jpeg(data['y'])
    z = tf.image.decode_jpeg(data['z'])
    weight = data['weight']
    
    ##frauas image's resolution are 480x640
    rgb = tf.reshape(rgb,(480,640,3))
    label = tf.reshape(label,(480,640,1))
    x = tf.reshape(x,(480,640,1))
    y = tf.reshape(y,(480,640,1))
    z = tf.reshape(z,(480,640,1))
    weight = tf.reshape(weight,[num_classes,])

    ##generate random crop coordinates
    fraction = tf.random.uniform([],0.7,1.0)
    height = tf.cast(480*fraction,tf.int32)
    width = tf.cast(640*fraction,tf.int32)
    y1 = tf.random.uniform([],0,481-height,dtype=tf.int32)
    x1 = tf.random.uniform([],0,641-width,dtype=tf.int32)

    ##crop out the patch
    rgb = crop(rgb,height,width,x1,y1)
    x = crop(x,height,width,x1,y1)
    y = crop(y,height,width,x1,y1)
    z = crop(z,height,width,x1,y1)
    label = crop(label,height,width,x1,y1)

    rgb = tf.cast(rgb,tf.float32)
    label = tf.cast(label,tf.float32)
    x = tf.cast(x,tf.float32)
    y = tf.cast(y,tf.float32)
    z = tf.cast(z,tf.float32)

    ##random flipping
    a = tf.random.uniform([],0.0,1.0)
    rgb = tf.cond(tf.math.greater(a,0.5),lambda: flip(rgb), lambda : rgb)
    x = tf.cond(tf.math.greater(a,0.5),lambda: flip(x), lambda : x)
    y = tf.cond(tf.math.greater(a,0.5),lambda: flip(y), lambda : y)
    z = tf.cond(tf.math.greater(a,0.5),lambda: flip(z), lambda : z)
    label = tf.cond(tf.math.greater(a,0.5),lambda: flip(label), lambda : label)
    x = tf.cond(tf.math.greater(a,0.5),lambda: subtract(x), lambda : x)

    ## adding random gaussian noises to points cloud
    noise_x = tf.random.normal(tf.shape(x), 0, 5)
    noise_y = tf.random.uniform(tf.shape(y), 0, 5)
    noise_z = tf.random.uniform(tf.shape(z), 0, 5)
    x = x+noise_x
    y = y+noise_y
    z = z+noise_z
   
   ## data augmentation
    saturation = tf.random.uniform([],0.5,3)
    contrast = tf.random.uniform([],0.5,2.5)
    brightness = tf.random.uniform([],-0.2,0.2)
    hue = tf.random.uniform([],-0.9,0.9)
    kernel_size = tf.random.uniform([],1,2,dtype=tf.int32)*2+1

    saturation = tf.squeeze(saturation)
    contrast = tf.squeeze(contrast)
    brightness = tf.squeeze(brightness)
    hue = tf.squeeze(hue)

    rgb = tf.image.adjust_brightness(rgb, brightness)
    rgb = tf.image.adjust_saturation(rgb, saturation)
    rgb = tf.image.adjust_contrast(rgb,contrast)
    blur_prob = tf.random.uniform([],0.0,0.7)
    rgb = tf.cond(tf.math.greater(blur_prob,0.5),lambda: gaussian_blur(rgb, kernel_size=kernel_size), lambda : rgb)
    
    ## creating penalizing weights (optional, might comment out)
    weight = tf.reshape(weight,(1,1,num_classes))
    label1 = tf.one_hot(tf.cast(label,tf.uint8),num_classes)
    label1 = tf.squeeze(label1)
    weight = tf.tile(weight,(tf.shape(x)[0],tf.shape(x)[1],1))
    weight = tf.math.multiply(weight,label1)
    weight = tf.reduce_sum(weight,axis=-1)
    weight = tf.reshape(weight,(tf.shape(x)[0],tf.shape(x)[1],1))

    rgb = rgb/255
    x = rescale(x)
    y = rescale(y)
    z = rescale(z)

    imgs = tf.concat([rgb,x,y,z],axis=-1)
    imgs = tf.image.resize(imgs,(480,640))

    label= tf.image.resize(label,(480,640))
    label = tf.cast(tf.math.round(label),tf.float32)
    weight = tf.image.resize(weight,(480,640))
    labels = tf.concat([label,weight],axis=-1)

    return imgs,labels

##parsing function for validation set
def val_parse(example_proto):
    data = tf.io.parse_single_example(example_proto, image_feature_description)
    rgb = tf.image.decode_jpeg(data['rgb'])
    label = tf.image.decode_png(data['label'],dtype=tf.uint8)
    x = tf.image.decode_jpeg(data['x'])
    y = tf.image.decode_jpeg(data['y'])
    z = tf.image.decode_jpeg(data['z'])
    weight = data['weight']
    
    rgb = tf.reshape(rgb,(480,640,3))
    label = tf.reshape(label,(480,640,1))
    x = tf.reshape(x,(480,640,1))
    y = tf.reshape(y,(480,640,1))
    z = tf.reshape(z,(480,640,1))
    weight = tf.reshape(weight,[num_classes,])

    weight = tf.reshape(weight,(1,1,num_classes))
    label1 = tf.one_hot(tf.cast(label,tf.uint8),num_classes)
    label1 = tf.squeeze(label1)
    weight = tf.tile(weight,(tf.shape(x)[0],tf.shape(x)[1],1))
    weight = tf.math.multiply(weight,label1)
    weight = tf.reduce_sum(weight,axis=-1)
    weight = tf.reshape(weight,(tf.shape(x)[0],tf.shape(x)[1],1))

    rgb = tf.cast(rgb,tf.float32)
    label = tf.cast(label,tf.float32)
    x = tf.cast(x,tf.float32)
    y = tf.cast(y,tf.float32)
    z = tf.cast(z,tf.float32)

    rgb = rgb/255
    x = rescale(x)
    y = rescale(y)
    z = rescale(z)

    imgs = tf.concat([rgb,x,y,z],axis=-1)
    imgs = tf.image.resize(imgs,(480,640))

    label= tf.image.resize(label,(480,640))
    label = tf.cast(tf.math.round(label),tf.float32)
    weight = tf.image.resize(weight,(480,640))
    labels = tf.concat([label,weight],axis=-1)
    
    return imgs,labels


##remember to upload your tfrecords onto google drive
train = tf.data.TFRecordDataset('train_custom.tfrecords').map(train_parse,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(400).batch(batch_size).prefetch(100).repeat()
val = tf.data.TFRecordDataset('val_custom.tfrecords').map(val_parse,num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().batch(1).prefetch(100)


In [ ]:
%cd /gdrive/My\ Drive/segmentation
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

l2=1e-4
dp=0.2

## toggle this if you want to train from scratch (without loading weights trained on 7class frauas dataset)
train_from_scratch = False 

##toggle these options for resume your previous training process (e.g. your session timeout or you restarted the kernel .....,)
##Don't toggle these option if you want to train from the weights pretrained on SceneNet. 
continue_training = False
if continue_training:
    starting_epoch = 0
    checkpoint_path = '' ## put the path of best checkpoint in the folder ./weights_custom/ here

## please change these options to True to unfreeze the encoders when the model no longer improves its accuracy
encoder_trainable_rgb=False 
encoder_trainable_depth = False 

## unfreeze encoders if train from scratch
if train_from_scratch:
    encoder_trainable_rgb=True 
    encoder_trainable_depth = True 

print('Users training config: ')
print('Train from scratch: ', train_from_scratch)
print('Continue training: ', continue_training)
print('Train RGB encoders: ', encoder_trainable_rgb)
print('Train Depth encoders: ', encoder_trainable_depth)

if continue_training:
    if checkpoint_path == '':
        print('user has not specified previous checkpoint path')
    else:
        print('weights will be restored from this checkpoint: ', checkpoint_path)

if (not continue_training) and (not train_from_scratch):
    print('Weights will be loaded from SceneNet checkpoint')


def get_link(layer, base_ch, growth_rate, grmul):
    if layer == 0:
        return base_ch, 0, []
    out_channels = growth_rate
    link = []
    
    for i in range(10):
        dv = 2 ** i
        if layer % dv == 0:
            k = layer - dv
            link.append(k)
            if i > 0:
                out_channels *= grmul
    
    out_channels = int(int(out_channels + 1) / 2) * 2
    
    return out_channels, link

def conv(x, depth, kernel_size=3, strides=1, padding = 'same',trainable=True):
    return tf.keras.layers.Conv2D(depth, (kernel_size,kernel_size), (strides,strides), padding= padding,
                                  kernel_initializer = tf.initializers.he_uniform(seed = 1),kernel_regularizer=tf.keras.regularizers.L2(l2),
                                  trainable=trainable)(x)

def batch_norm(x, trainable=True):
    return tf.keras.layers.BatchNormalization(fused=True,trainable=trainable)(x)

def relu(x):
    return tf.nn.relu(x)

def resize(x, size):
    return tf.image.resize(x, size)

def drop_out(x):
    return tf.keras.layers.Dropout(dp)(x)

def avg_pool(x, pool_size = 2, strides = 2):
    return tf.keras.layers.AveragePooling2D(pool_size, strides, padding = 'same')(x)

def conv_layer(x, depth, kernel_size=3, strides=1,trainable=True,dropout=True):
    x = conv(x, depth = depth, kernel_size=kernel_size, strides=strides,trainable=trainable)
    x = batch_norm(x)
    x = relu(x)
    if dropout:
        x = drop_out(x)   
    return x

def hardblock(x, in_channels, growth_rate, grmul, n_layers,trainable=True,dropout=True):
    layers = [x]
    out_layers = []
    out_channels = 0
    
    for i in range(n_layers):    
        links = []
        out_ch, link = get_link(i+1, in_channels, growth_rate, grmul)
    
        for i in link:
            links.append(layers[i])

        x = tf.concat(links, axis = -1)
        layers.append(conv_layer(x, out_ch,trainable=trainable,dropout=dropout))
    
    for i in range(len(layers)):
        if (i == len(layers)-1) or (i%2 == 1):
            out_layers.append(layers[i])  
    x = tf.concat(out_layers, axis = -1)
    return x

def transition_up(x, skip_connection):
    x = resize(x, (tf.shape(skip_connection)[1], tf.shape(skip_connection)[2]))
    x = tf.concat([x, skip_connection], axis = -1)
    return x

def se_block(x,trainable=True):
    num_reduced_filters = tf.cast(x.shape[-1]*0.25,tf.int32).numpy()
    se_tensor = layers.GlobalAveragePooling2D()(x)
    se_tensor = layers.Reshape((1, 1, x.shape[-1]))(se_tensor)
    se_tensor = layers.Conv2D(num_reduced_filters, 1, activation=tf.keras.activations.swish, padding='same',trainable=trainable,
                              kernel_initializer = tf.initializers.he_uniform(seed = 1),kernel_regularizer=tf.keras.regularizers.L2(l2))(se_tensor)
    se_tensor = layers.Conv2D(x.shape[-1],1,activation='sigmoid',padding='same',trainable=trainable,
                              kernel_initializer = tf.initializers.he_uniform(seed = 1),kernel_regularizer=tf.keras.regularizers.L2(l2))(se_tensor)
    x = layers.multiply([x, se_tensor])   
    return x

def hardnet(img): 
    depth = img[:,:,:,3:]
    img = img[:,:,:,:3]
    first_ch  = np.asarray([16,24,32,48])
    depth_conv1 = np.asarray([64, 96, 160, 224, 320])
    grmul = 1.7
    gr       = np.asarray([  10,16,18,24,32])
    n_layers = [4, 4, 8, 8, 8]
    up_sample = np.asarray([126,238,374,534])
    skip_connections = []
    skip_connections_d = []
    blks = len(n_layers)
    
    x = img
    x = conv_layer(x, first_ch[0], strides = 2,trainable=encoder_trainable_rgb)
    x = conv_layer(x, first_ch[1],trainable=encoder_trainable_rgb)
    x = conv_layer(x, first_ch[2], strides = 2,trainable=encoder_trainable_rgb)
    x = conv_layer(x, first_ch[3],trainable=encoder_trainable_rgb)

    
    x_d = depth
    x_d = conv_layer(x_d, first_ch[0], strides = 2,trainable=encoder_trainable_depth)
    x_d = conv_layer(x_d, first_ch[1],trainable=encoder_trainable_depth)
    x_d = conv_layer(x_d, first_ch[2], strides = 2,trainable=encoder_trainable_depth)
    x_d = conv_layer(x_d, first_ch[3],trainable=encoder_trainable_depth)
    
    ch = first_ch[3]
    ch_d = first_ch[3]
    for i in range(blks):
        x = hardblock(x, ch, gr[i], grmul, n_layers[i],trainable=encoder_trainable_rgb)
        x_d = hardblock(x_d, ch_d, gr[i], grmul, n_layers[i],trainable=encoder_trainable_depth)
        
        if i < blks-1:
            skip_connections.append(tf.math.add(se_block(x,trainable=encoder_trainable_rgb),se_block(x_d,trainable=encoder_trainable_depth))) 
        x = conv_layer(x, x.shape[-1], kernel_size = 1,trainable=encoder_trainable_rgb)
        x_d = conv_layer(x_d, x_d.shape[-1], kernel_size = 1,trainable=encoder_trainable_depth)
        if i < blks-1:
            x = avg_pool(x)
            x_d = avg_pool(x_d)
        ch = x.shape[-1]
        ch_d = x_d.shape[-1]
    
    x = tf.math.add(se_block(x,trainable=encoder_trainable_rgb),se_block(x_d,trainable=encoder_trainable_depth))

    n_blocks = blks-1
    for i in range(n_blocks-1,-1,-1):
        skip = skip_connections.pop()
        x = transition_up(x, skip)

        cur_channels_count = x.shape[-1]//2

        x = conv_layer(x, cur_channels_count, kernel_size=1,dropout=False)
        x = hardblock(x, cur_channels_count, gr[i], grmul, n_layers[i],dropout=False)

    x_pre_conv = x
    x = conv(x, depth = 7, kernel_size = 1)
    
    return x,x_pre_conv

##first we have to built the model for scenenet to load the pretrained weights
input = tf.keras.layers.Input(shape=(480,640,6))
output1,output2 = hardnet(input)
model = tf.keras.Model(inputs=input, outputs=[output1,output2])


In [ ]:
if (not train_from_scratch) and (not continue_training): 
    ## make sure you have uploaded the pretrained file weights_scenenet.h5 in the same folder with this script
    model.load_weights('./weights_frauas_7classes.h5')

In [ ]:
##now we build the model for your custom dataset

x = conv(output2,depth=num_classes,kernel_size=1)
x = resize(x, (480,640))
model = tf.keras.Model(inputs=input, outputs=x)

In [ ]:
import tensorflow as tf
import numpy as np

class IoU(tf.keras.metrics.Metric):
    def __init__(self, name="IoU", **kwargs):
        super(IoU, self).__init__(name=name, **kwargs)
        self.m = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred,sample_weight=None):
        y_true = tf.expand_dims(y_true[:,:,:,0],axis=-1)
        y_pred = tf.expand_dims(tf.math.argmax(y_pred,axis=-1),axis=-1)
        self.m.update_state(y_true,y_pred)

    def result(self):
        return self.m.result()

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.m.reset_states()

def custom_loss(y_true,y_pred):
    weight = y_true[:,:,:,1]
    y_true = y_true[:,:,:,0]
    weight = tf.expand_dims(weight,axis=-1)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #loss1 = loss(y_true, y_pred, sample_weight=weight) #you may play around by uncommenting this
    loss1 = loss(y_true, y_pred)

    prob = tf.nn.softmax(y_pred)
    y_true = tf.one_hot(tf.cast(y_true,tf.uint8),num_classes)
    num = tf.reduce_sum(prob*y_true,axis=-1)
    denom = tf.reduce_sum(prob + y_true-prob*y_true,axis=-1)+1e-7
    loss2 = 1 - num/denom
    loss2 = tf.expand_dims(loss2,axis=-1)
    #loss2 = tf.reduce_mean(loss2*weight) #you may play around by uncommenting this
    loss2 = tf.reduce_mean(loss2)

    return loss1 + loss2

##learning rate scheduler
def scheduler(epoch):
    epoch = epoch+starting_epoch
    if 0.002/(0.2*epoch+1) > 0.0001:
        return 0.002/(0.2*epoch+1)
    else:
        return 0.0001
        
##configure the model training and validation policies
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss=custom_loss, metrics=custom_metrics)

##load previous model checkpoint if you want to resume training or if you want to unfreeze the encoders
if continue_training: 
    model.load_weights(checkpoint_path)

##save only best checkpoints
checkpoint_filepath = './weight_custom/loss:{loss:.2f}_valIoU:{val_custom_metrics:.2f}.h5'
cb_list = [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,save_best_only = True, monitor = 'val_custom_metrics',verbose=1,mode='max'),
        tf.keras.callbacks.LearningRateScheduler(scheduler)]
model.fit(train,epochs = 1000,steps_per_epoch=1500,validation_data=val,callbacks=cb_list,verbose=1)


In [ ]:
best_model_path = '' #choose the best model in the folder weights_custom
model.load_weights(best_model_path)
vals = iter(val) ##create an iterator for the validation set


In [ ]:
## code snippet for visualization

imgs,labels = next(vals)
output = model.predict_on_batch(imgs)
output = output[0]
output = np.argmax(output,axis=-1)
output = loutput
label = labeller(labels[0,:,:,0])
rgb = imgs[0,:,:,:3]
def plotter(img):
    plt.imshow(img)
    if img.shape[-1]!=3:
        plt.colorbar()
    plt.show()

plotter(output)
plotter(label)
plotter(rgb)

In [ ]:
## code snippet to calculate the model performance

def numpy_metrics(y_pred, y_true, n_classes):
    y_pred = np.argmax(y_pred, axis=-1).flatten()
    y_true = y_true.flatten()
    I = np.zeros(n_classes)
    U = np.zeros(n_classes)

    for i in range(n_classes):
        y_true_i = y_true == i
        y_pred_i = y_pred == i

        I[i] = np.sum(y_true_i & y_pred_i)
        U[i] = np.sum((y_true_i | y_pred_i))

    accuracy = np.sum(I) / image_size
    return I, U, accuracy

acc_tot = 0
n_imgs = 0
I_tot = 0
U_tot = 0
vals =iter(val)
while 1:
    try:
        imgs,labels = next(vals)
        output = model.predict_on_batch(imgs)
        output = output[0]
        gt = labels[0,:,:,0].numpy()
        I, U, acc = numpy_metrics(output, gt, num_classes)
        I_tot += I
        U_tot += U
        acc_tot += acc 
        n_imgs += 1
    except:
        print('finish')
        break

iou= np.mean(I_tot / (U_tot+1e-7))
accuracy = acc_tot / n_imgs
for label, jacc in zip(range(7), I_tot / U_tot):
    print('class {} :\t{:.4f}'.format(label, jacc)) ## please have a look at the mapping in the json_to_label file
print ('Mean Jaccard', iou)
print ('Mean accuracy', accuracy)